In [1]:
import os
import zipfile
import pandas as pd
import basedosdados as bd

INPUT = os.path.join(os.getcwd(), "input")
OUTPUT = os.path.join(os.getcwd(), "output")

os.makedirs(INPUT, exist_ok=True)
os.makedirs(OUTPUT, exist_ok=True)

In [2]:
def read_sheet(sheet_name: str, skiprows: int = 8) -> pd.DataFrame:
    return pd.read_excel(
        os.path.join(
            INPUT,
            "tx_rend_brasil_regioes_ufs_esp.xlsx"
        ),
        skiprows=skiprows,
        sheet_name=sheet_name
    )

In [3]:
excel_data = pd.ExcelFile(os.path.join(
            INPUT,
            "tx_rend_brasil_regioes_ufs_esp.xlsx"
        ))

# Get the sheet names
sheet_names = excel_data.sheet_names

In [4]:
dfs = {
    sheet_name: read_sheet(sheet_name)
    for sheet_name in sheet_names
}

In [5]:
dfs

{'BRASIL_REGIOES_UFS ':                                    NU_ANO_CENSO           UNIDGEO  \
 0                                          2007            Brasil   
 1                                          2007             Norte   
 2                                          2007          Nordeste   
 3                                          2007           Sudeste   
 4                                          2007               Sul   
 ..                                          ...               ...   
 492                                        2021       Mato Grosso   
 493                                        2021             Goiás   
 494                                        2021  Distrito Federal   
 495                                         NaN               NaN   
 496  Fonte: Censo da Educação Básica 2021/INEP.               NaN   
 
     NO_CATEGORIA NO_DEPENDENCIA  1_CAT_FUN  1_CAT_FUN_AI  1_CAT_FUN_AF  \
 0          Total          Total       73.1          71.6   

In [6]:
for sheet_name, df in dfs.items():
    print(f"Sheet: {sheet_name}")
    print(df.columns)  # This will print the column names of each DataFrame
    print()  # Adds a blank line for readability


Sheet: BRASIL_REGIOES_UFS 
Index(['NU_ANO_CENSO', 'UNIDGEO', 'NO_CATEGORIA', 'NO_DEPENDENCIA',
       '1_CAT_FUN', '1_CAT_FUN_AI', '1_CAT_FUN_AF', '1_CAT_MED', '2_CAT_FUN',
       '2_CAT_FUN_AI', '2_CAT_FUN_AF', '2_CAT_MED', '3_CAT_FUN',
       '3_CAT_FUN_AI', '3_CAT_FUN_AF', '3_CAT_MED'],
      dtype='object')



In [7]:
RENAME_COLUMNS = {
    'NU_ANO_CENSO':'ano', 
    'UNIDGEO':'nome_uf',
    '1_CAT_FUN_AI':'taxaaprovacao_anosiniciais', 
    '1_CAT_FUN_AF':'taxaaprovacao_anosfinais', 
    '1_CAT_MED':'taxaaprovacao_ensinomedio', 
    '2_CAT_FUN_AI':'taxareprovacao_anosiniciais', 
    '2_CAT_FUN_AF':'taxareprovacao_anosfinais', 
    '2_CAT_MED':'taxareprovacao_ensinomedio', 
    '3_CAT_FUN_AI':'taxaabandono_anosiniciais', 
    '3_CAT_FUN_AF':'taxaabandono_anosfinais', 
    '3_CAT_MED' : 'taxaabandono_ensinomedio' 
}

In [8]:
def drop_unused_columns(df: pd.DataFrame) -> pd.DataFrame:
    cols_drop = [
        col
        for col in df.columns
        if col.startswith("NO_") 
        or col.startswith("1_") 
        or col.startswith("2_") 
        or col.startswith("3_")
    ]

    return df.drop(columns=cols_drop)

dfs = {
    name: drop_unused_columns(
        df.rename(columns=RENAME_COLUMNS, errors="raise")
    )
    for name, df in dfs.items()
}

In [9]:
dfs

{'BRASIL_REGIOES_UFS ':                                             ano           nome_uf  \
 0                                          2007            Brasil   
 1                                          2007             Norte   
 2                                          2007          Nordeste   
 3                                          2007           Sudeste   
 4                                          2007               Sul   
 ..                                          ...               ...   
 492                                        2021       Mato Grosso   
 493                                        2021             Goiás   
 494                                        2021  Distrito Federal   
 495                                         NaN               NaN   
 496  Fonte: Censo da Educação Básica 2021/INEP.               NaN   
 
      taxaaprovacao_anosiniciais  taxaaprovacao_anosfinais  \
 0                          71.6                      78.7   
 1         

In [10]:
for sheet_name, df in dfs.items():
    print(f"Sheet: {sheet_name}")
    print(df.columns)  # This will print the column names of each DataFrame
    print()  # Adds a blank line for readability

Sheet: BRASIL_REGIOES_UFS 
Index(['ano', 'nome_uf', 'taxaaprovacao_anosiniciais',
       'taxaaprovacao_anosfinais', 'taxaaprovacao_ensinomedio',
       'taxareprovacao_anosiniciais', 'taxareprovacao_anosfinais',
       'taxareprovacao_ensinomedio', 'taxaabandono_anosiniciais',
       'taxaabandono_anosfinais', 'taxaabandono_ensinomedio'],
      dtype='object')



In [11]:
melted_dataframe = pd.concat(
    [
        df.pipe(
            lambda d: d.loc[(d["nome_uf"] == 'Brasil' )]
        )
        .pipe(
            lambda d: pd.melt(
                d,
                id_vars=["ano", "nome_uf"],
                value_vars=d.columns.difference(["ano", "nome_uf"]).tolist(),  # Convert to list
                var_name="metrica",
                value_name="valor",
            )
        )
    ]
)

In [12]:
melted_dataframe

,ano,nome_uf,metrica,valor
0,2007,Brasil,taxaabandono_anosfinais,5.1
1,2008,Brasil,taxaabandono_anosfinais,5.0
2,2009,Brasil,taxaabandono_anosfinais,4.3
3,2010,Brasil,taxaabandono_anosfinais,4.6
4,2011,Brasil,taxaabandono_anosfinais,4.6
...,...,...,...,...
130,2017,Brasil,taxareprovacao_ensinomedio,11.2
131,2018,Brasil,taxareprovacao_ensinomedio,11.4
132,2019,Brasil,taxareprovacao_ensinomedio,9.4
133,2020,Brasil,taxareprovacao_ensinomedio,2.2


In [13]:
melted_dataframe['etapa_ensino'] = melted_dataframe['metrica'].apply(
    lambda v: v.split('_')[-1])  # Extracts 'anosiniciais', 'anosfinais', or 'ensinomedio'
melted_dataframe['tipo_metrica'] = melted_dataframe['metrica'].apply(
    lambda v: v.split('_')[0])    # Extracts 'taxaaprovacao', 'taxareprovacao', 'taxaabandono'

# Pivoting the melted DataFrame to get desired columns
df_final = melted_dataframe.pivot_table(index=['ano', 'nome_uf', 'etapa_ensino'], 
                                 columns='tipo_metrica', 
                                 values='valor').reset_index()

In [14]:
RENAME_COLUMNS_MELTED = {
    'taxaabandono':'taxa_abandono', 
    'taxaaprovacao':'taxa_aprovacao',
    'taxareprovacao':'taxa_reprovacao'   
}

etapa_ensino = {
    'anosiniciais': 'Ensino Fundamental – Anos Iniciais',
    'anosfinais':'Ensino Fundamental – Anos Finais',
    'ensinomedio':'Ensino Médio Regular'
}

In [15]:
df_final

tipo_metrica,ano,nome_uf,etapa_ensino,taxaabandono,taxaaprovacao,taxareprovacao
0,2007,Brasil,anosfinais,5.1,78.7,16.2
1,2007,Brasil,anosiniciais,4.6,71.6,23.8
2,2007,Brasil,ensinomedio,7.4,79.6,13.0
3,2008,Brasil,anosfinais,5.0,78.8,16.2
4,2008,Brasil,anosiniciais,4.5,72.4,23.1
5,2008,Brasil,ensinomedio,7.8,79.3,12.9
6,2009,Brasil,anosfinais,4.3,81.5,14.2
7,2009,Brasil,anosiniciais,3.3,74.4,22.3
8,2009,Brasil,ensinomedio,7.0,80.0,13.0
9,2010,Brasil,anosfinais,4.6,81.1,14.3


In [16]:
df_final = df_final.rename(columns=RENAME_COLUMNS_MELTED)
df_final['etapa_ensino'] = df_final['etapa_ensino'].replace(etapa_ensino)

In [17]:

# Rename the 'sigla' column to 'sigla_uf' and drop the 'nome' column
df_final = df_final.drop(columns=['nome_uf'])


In [18]:
df_final = df_final[['ano', 'etapa_ensino', 'taxa_aprovacao','taxa_reprovacao','taxa_abandono']]

In [19]:
df_final

tipo_metrica,ano,etapa_ensino,taxa_aprovacao,taxa_reprovacao,taxa_abandono
0,2007,Ensino Fundamental – Anos Finais,78.7,16.2,5.1
1,2007,Ensino Fundamental – Anos Iniciais,71.6,23.8,4.6
2,2007,Ensino Médio Regular,79.6,13.0,7.4
3,2008,Ensino Fundamental – Anos Finais,78.8,16.2,5.0
4,2008,Ensino Fundamental – Anos Iniciais,72.4,23.1,4.5
5,2008,Ensino Médio Regular,79.3,12.9,7.8
6,2009,Ensino Fundamental – Anos Finais,81.5,14.2,4.3
7,2009,Ensino Fundamental – Anos Iniciais,74.4,22.3,3.3
8,2009,Ensino Médio Regular,80.0,13.0,7.0
9,2010,Ensino Fundamental – Anos Finais,81.1,14.3,4.6


In [20]:
path = os.path.join(
        OUTPUT, "educacao_especial_brasil_taxa_rendimento"
    )

os.makedirs(path, exist_ok=True)
df_final.astype(str).to_csv(os.path.join(path, "data.csv"), index=False)